라이브러리들

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage,HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate,ChatPromptTemplate,load_prompt,MessagesPlaceholder
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache,set_debug
from langchain.cache import InMemoryCache
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory,ConversationSummaryBufferMemory,ConversationSummaryMemory,ConversationKGMemory
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough,RunnableLambda
from langchain.document_loaders import TextLoader,UnstructuredFileLoader #<< 이 녀석은 pdf와 같은 비정형? 파일을 로드하는 녀석임
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import Chroma,FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA





자주 쓰는 변수들

In [2]:
CHAT = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    # model = "gpt-4o-mini",
)

#3.0

In [3]:
a = CHAT.predict("배고픈데 먹을 거 추천좀. 되도록이면 아시아 음식을 추천해줘. 일식이랑 한식으로 추천해줘")
# 예제에서 특징은 chat이라는 변수에 어떠한 모델을 사용하든, predict라는 메소드 사용 가능 / 답변은 매번 바뀜

a

일식으로는 초밥이나 라멘을 추천드리고, 한식으로는 비빔밥이나 불고기를 추천해드립니다. 맛있게 드시고 배부르게 채워보세요!

'일식으로는 초밥이나 라멘을 추천드리고, 한식으로는 비빔밥이나 불고기를 추천해드립니다. 맛있게 드시고 배부르게 채워보세요!'

#3.1

In [19]:
message = [
    SystemMessage(
        content="넌 이제부터 매우 awsome하고 coooool한 음식의 specialist야. 그 어떠한 음식에 대해 물어보면 재료와 레시피에 대해 말할 수 있어."),
    AIMessage(
        content="안녕하세요. 저는 맛있는 녀석입니다."),
    HumanMessage(
        content="일식 덮밥집에서 파는 부타동에 대해 설명해줘.")
]

CHAT.predict_messages(message)

부타동(豚丼)은 일본의 전통적인 덮밥 요리로, 주로 돼지고기를 사용하여 만드는 맛있는 요리입니다. 부타동은 간장, 미림, 설탕 등을 이용해 달콤하고 짭짤한 소스를 만들어 돼지고기를 조리한 후, 밥 위에 얹어 제공합니다. 

### 재료:
- 밥 (일본식 짧은 쌀) 2인분
- 돼지고기 (삼겹살 또는 목살) 300g
- 양파 1개
- 대파 (선택 사항) 약간
- 간장 3큰술
- 미림 2큰술
- 설탕 1큰술
- 식용유 약간
- 깨소금 (선택 사항)

### 레시피:
1. **재료 준비**: 돼지고기는 얇게 썰고, 양파는 채 썰어 준비합니다. 대파는 송송 썰어 놓습니다.

2. **소스 만들기**: 작은 그릇에 간장, 미림, 설탕을 넣고 잘 섞어 소스를 만듭니다.

3. **팬에 굽기**: 중불로 달군 팬에 식용유를 두르고, 양파를 넣어 볶아줍니다. 양파가 투명해지면 돼지고기를 추가합니다.

4. **조리하기**: 돼지고기가 익으면 미리 만든 소스를 팬에 부어줍니다. 소스가 고기에 잘 스며들도록 볶아가며 조리합니다. 소스가 약간 졸아들 때까지 볶아주세요.

5. **밥 위에 얹기**: 따뜻한 밥을 그릇에 담고, 그 위에 조리한 돼지고기와 양파를 올립니다. 원한다면 대파와 깨소금을 뿌려 장식합니다.

6. **서빙**: 뜨거운 부타동을 바로 서빙하고, 취향에 따라 추가로 간장을 뿌려 먹으면 더욱 맛있습니다.

부타동은 간단하면서도 깊은 맛을 즐길 수 있는 요리로, 집에서도 쉽게 만들 수 있습니다. 맛있게 드세요!

AIMessageChunk(content='부타동(豚丼)은 일본의 전통적인 덮밥 요리로, 주로 돼지고기를 사용하여 만드는 맛있는 요리입니다. 부타동은 간장, 미림, 설탕 등을 이용해 달콤하고 짭짤한 소스를 만들어 돼지고기를 조리한 후, 밥 위에 얹어 제공합니다. \n\n### 재료:\n- 밥 (일본식 짧은 쌀) 2인분\n- 돼지고기 (삼겹살 또는 목살) 300g\n- 양파 1개\n- 대파 (선택 사항) 약간\n- 간장 3큰술\n- 미림 2큰술\n- 설탕 1큰술\n- 식용유 약간\n- 깨소금 (선택 사항)\n\n### 레시피:\n1. **재료 준비**: 돼지고기는 얇게 썰고, 양파는 채 썰어 준비합니다. 대파는 송송 썰어 놓습니다.\n\n2. **소스 만들기**: 작은 그릇에 간장, 미림, 설탕을 넣고 잘 섞어 소스를 만듭니다.\n\n3. **팬에 굽기**: 중불로 달군 팬에 식용유를 두르고, 양파를 넣어 볶아줍니다. 양파가 투명해지면 돼지고기를 추가합니다.\n\n4. **조리하기**: 돼지고기가 익으면 미리 만든 소스를 팬에 부어줍니다. 소스가 고기에 잘 스며들도록 볶아가며 조리합니다. 소스가 약간 졸아들 때까지 볶아주세요.\n\n5. **밥 위에 얹기**: 따뜻한 밥을 그릇에 담고, 그 위에 조리한 돼지고기와 양파를 올립니다. 원한다면 대파와 깨소금을 뿌려 장식합니다.\n\n6. **서빙**: 뜨거운 부타동을 바로 서빙하고, 취향에 따라 추가로 간장을 뿌려 먹으면 더욱 맛있습니다.\n\n부타동은 간단하면서도 깊은 맛을 즐길 수 있는 요리로, 집에서도 쉽게 만들 수 있습니다. 맛있게 드세요!')

#3.2

In [20]:
prom_template = PromptTemplate.from_template("{place}을(를) 가는데, 추천 좀 해줘봐")
#PromptTemplate는 단순 메세지를 프롬프트로 사용 할 때 유용함

prom_template = prom_template.format(place = "맘스터치")

CHAT.predict(prom_template)

맘스터치에 가신다면 추천할 메뉴는 다음과 같습니다:

1. **싸이순살**: 바삭한 튀김옷과 함께 촉촉한 치킨이 일품입니다. 간편하게 먹기 좋고, 양념도 다양해서 선택할 수 있어요.

2. **싸이너겟**: 간편하게 먹을 수 있는 너겟으로, 아이들 간식이나 술안주로도 좋습니다. 다양한 소스와 함께 즐길 수 있어요.

3. **핫윙**: 매콤한 맛을 좋아하신다면 핫윙을 추천합니다. 바삭한 식감과 매콤한 양념이 잘 어울립니다.

4. **감자튀김**: 맘스터치의 감자튀김은 바삭하고 고소해서 사이드로 좋습니다. 치킨과 함께 드시면 더욱 맛있어요.

5. **소스**: 다양한 소스가 준비되어 있으니, 자신이 좋아하는 소스와 함께 즐겨보세요.

원하시는 스타일에 따라 선택하시면 좋을 것 같습니다! 맛있게 드세요!

'맘스터치에 가신다면 추천할 메뉴는 다음과 같습니다:\n\n1. **싸이순살**: 바삭한 튀김옷과 함께 촉촉한 치킨이 일품입니다. 간편하게 먹기 좋고, 양념도 다양해서 선택할 수 있어요.\n\n2. **싸이너겟**: 간편하게 먹을 수 있는 너겟으로, 아이들 간식이나 술안주로도 좋습니다. 다양한 소스와 함께 즐길 수 있어요.\n\n3. **핫윙**: 매콤한 맛을 좋아하신다면 핫윙을 추천합니다. 바삭한 식감과 매콤한 양념이 잘 어울립니다.\n\n4. **감자튀김**: 맘스터치의 감자튀김은 바삭하고 고소해서 사이드로 좋습니다. 치킨과 함께 드시면 더욱 맛있어요.\n\n5. **소스**: 다양한 소스가 준비되어 있으니, 자신이 좋아하는 소스와 함께 즐겨보세요.\n\n원하시는 스타일에 따라 선택하시면 좋을 것 같습니다! 맛있게 드세요!'

In [21]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "넌 이제부터 매우 awsome하고 coooool한 {job}의 specialist야. 그 어떠한 질문이든 대답 할 수 있어."),
    ("ai", "안녕하세요. 저는 {name}입니다."),
    ("human", "{place}을(를) 가는데, 추천 좀 해줘."),
])
#ChatPromptTemplate는 조금 더 정교하게 설정된 프롬프트를 탬플릿을 통해 사용 할 때 유용함


chat_template = chat_template.format(job="음식",name="맛있는 녀석",place="맘스터치")

CHAT.predict(chat_template)

맘스터치에 가신다면, 꼭 먹어봐야 할 메뉴는 "싸이순살"과 "싸이너겟"입니다! 싸이순살은 바삭한 튀김옷과 함께 부드러운 치킨이 조화를 이루고, 싸이너겟은 간편하게 먹기 좋은 사이드 메뉴로 정말 맛있어요. 또한, 맘스터치의 특제 소스인 "매운 소스"를 곁들이면 더욱 맛있답니다. 그리고 사이드로는 "감자튀김"이나 "콜라"를 추천해요. 배고프게 드세요!

'맘스터치에 가신다면, 꼭 먹어봐야 할 메뉴는 "싸이순살"과 "싸이너겟"입니다! 싸이순살은 바삭한 튀김옷과 함께 부드러운 치킨이 조화를 이루고, 싸이너겟은 간편하게 먹기 좋은 사이드 메뉴로 정말 맛있어요. 또한, 맘스터치의 특제 소스인 "매운 소스"를 곁들이면 더욱 맛있답니다. 그리고 사이드로는 "감자튀김"이나 "콜라"를 추천해요. 배고프게 드세요!'

#3.3

In [22]:
#BaseOutputParser를 사용하여 결과 출력을 parse하기

class CommaOutParser(BaseOutputParser):
    def parse(self,text): # 필수 메소드
        items = text.strip().split(".")
        return(list(map(str.strip,items)))
    
parser = CommaOutParser()
parser.parse("  안녕하세요     .   유령개입니다 .    잘부탁드리겠습니다.")

#3분 24초

['안녕하세요', '유령개입니다', '잘부탁드리겠습니다', '']

In [23]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문이 들어오면, 최대 {n}개만큼의 리스트로 대답하는 리스트 대답 기계입니다. 리스트가 아닌걸론 대답하지 마세요. 그리고, 각 리스트들은 ,(콤마)로 구분되야해요."),
    ("human", "{question}"),
])

chat_template = chat_template.format(
    n = 10,
    question = "아시아에 위치한 나라를 알려주세요.  "
)

rs = CHAT.predict(chat_template)
parser.parse(rs)

중국, 일본, 인도, 한국, 베트남, 태국, 인도네시아, 말레이시아, 필리핀, 방글라데시

['중국, 일본, 인도, 한국, 베트남, 태국, 인도네시아, 말레이시아, 필리핀, 방글라데시']

#-------------------------

In [24]:
# 위의 두개와 달리, 더 간단하게 할 수 있는 버전이 있다. 여기선 오직 CHAT이라는 chatmodel 변수와, template. parser 세 개만이 존재한다.

class CommaOutParser(BaseOutputParser):
    def parse(self,text): # 필수 메소드
        items = text.strip().split(".")
        return(list(map(str.strip,items)))

chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문이 들어오면, 최대 {n}개만큼의 리스트로 대답하는 리스트 대답 기계입니다. 리스트가 아닌걸론 대답하지 마세요. 그리고, 각 리스트들은 ,(콤마)로 구분되야해요."),
    ("human", "{question}"),
])


chain = chat_template | CHAT | CommaOutParser()
chain.invoke({
    "n":3,
    "question":"유럽의 나라"
})

프랑스, 독일, 이탈리아

['프랑스, 독일, 이탈리아']

#3.4

In [25]:
chef_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계적인 슈퍼 클래스의 요리사입니다. 어떠한 요리를 요구하든 그 요리의 재료와 레시피에 대해 알려줄 수 있습니다."),
    ("human", "저는 {food} 요리를 만들고 싶습니다. 들어가는 재료와 레시피에 대해 알려주실 수 있을까요?"),
])
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 어떠한 요리의 레시피가 들어와도 채식으로 변형시킬 수 있는 세계 대표의 채식 요리사입니다. 레시피가 들어오면, 해당 메뉴의 재료를 채식에 맞게 변형시켜야합니다."),
    ("human", "{recipe}"),
])

chef_chain = chef_template | CHAT
veg_chef_chain = veg_chef_template | CHAT
fianal_chain = {"recipe":chef_chain} | veg_chef_chain # chef_template | CHAT | veg_chef_template | CHAT << 이것과 같은 형태임.
# 여러개의 체인을 위와 같은 방식으로 엮을 수 있음. 이건 머신러닝과 엇비슷한 형태라고 볼 수 있음

fianal_chain.invoke({
    "food": "프랑스"
})


물론입니다! 프랑스 요리는 다양하고 맛있습니다. 여기서는 클래식한 프랑스 요리인 **부야베스(해산물 스튜)**를 소개하겠습니다.

### 부야베스 재료
**재료:**
- 생선(흰살 생선, 예: 대구, 광어) 500g
- 조개(홍합, 바지락 등) 300g
- 새우 200g
- 오징어 200g
- 양파 1개
- 샬롯 1개
- 마늘 3쪽
- 토마토 2개 (또는 토마토 통조림 400g)
- 생강 1 작은 조각
- 사프란 1 작은 술
- 올리브 오일 4 큰 술
- 화이트 와인 200ml
- 생선 육수 1리터
- 파슬리 (다진 것) 약간
- 소금, 후추 (취향에 따라)
- 바게트 (서빙용)

### 레시피
1. **재료 준비하기**: 생선과 해산물은 깨끗이 손질하고, 양파, 샬롯, 마늘은 다지고, 토마토는 껍질을 벗기고 잘게 썰어줍니다.

2. **기름에 볶기**: 큰 냄비에 올리브 오일을 두르고, 다진 양파, 샬롯, 마늘을 넣고 중불에서 볶아줍니다. 양파가 투명해질 때까지 볶습니다.

3. **토마토와 향신료 추가하기**: 썰어놓은 토마토와 생강, 사프란을 냄비에 넣고 잘 섞어줍니다. 5분 정도 더 볶아주세요.

4. **육수와 화이트 와인 추가하기**: 생선 육수와 화이트 와인을 냄비에 붓고, 끓입니다. 끓기 시작하면 불을 줄이고 15분 정도 끓입니다.

5. **해산물 넣기**: 준비한 생선, 조개, 새우, 오징어를 넣고 소금과 후추로 간을 맞춥니다. 해산물이 익을 때까지 약 10분 정도 더 끓입니다.

6. **서빙하기**: 부야베스가 완성되면 그릇에 담고, 다진 파슬리를 뿌린 후 바게트와 함께 서빙합니다.

부야베스는 풍부한 해산물의 맛이 어우러져 매우 맛있습니다. 좋은 식사 되세요!물론입니다! 클래식한 프랑스 요리인 부야베스를 채식 버전으로 변형해 보겠습니다.

### 채식 부야베스 재료
**재료:**
- 흰살 생선 대체품 (예: 두부, 템페, 또는 식물성 생선 대체품) 500g
- 버섯 (예: 표고버섯, 느타리버섯 등) 300g
- 완두콩 (또는 다른 콩류)

AIMessageChunk(content='물론입니다! 클래식한 프랑스 요리인 부야베스를 채식 버전으로 변형해 보겠습니다.\n\n### 채식 부야베스 재료\n**재료:**\n- 흰살 생선 대체품 (예: 두부, 템페, 또는 식물성 생선 대체품) 500g\n- 버섯 (예: 표고버섯, 느타리버섯 등) 300g\n- 완두콩 (또는 다른 콩류) 200g\n- 애호박 200g\n- 양파 1개\n- 샬롯 1개\n- 마늘 3쪽\n- 토마토 2개 (또는 토마토 통조림 400g)\n- 생강 1 작은 조각\n- 사프란 1 작은 술\n- 올리브 오일 4 큰 술\n- 화이트 와인 200ml\n- 채소 육수 1리터\n- 파슬리 (다진 것) 약간\n- 소금, 후추 (취향에 따라)\n- 바게트 (서빙용)\n\n### 레시피\n1. **재료 준비하기**: 두부나 템페는 깍둑썰기로 자르고, 버섯과 애호박은 적당한 크기로 썰고, 양파, 샬롯, 마늘은 다집니다. 토마토는 껍질을 벗기고 잘게 썰어줍니다.\n\n2. **기름에 볶기**: 큰 냄비에 올리브 오일을 두르고, 다진 양파, 샬롯, 마늘을 넣고 중불에서 볶아줍니다. 양파가 투명해질 때까지 볶습니다.\n\n3. **토마토와 향신료 추가하기**: 썰어놓은 토마토와 생강, 사프란을 냄비에 넣고 잘 섞어줍니다. 5분 정도 더 볶아주세요.\n\n4. **육수와 화이트 와인 추가하기**: 채소 육수와 화이트 와인을 냄비에 붓고, 끓입니다. 끓기 시작하면 불을 줄이고 15분 정도 끓입니다.\n\n5. **재료 넣기**: 준비한 두부 또는 템페, 버섯, 완두콩, 애호박을 넣고 소금과 후추로 간을 맞춥니다. 재료가 익을 때까지 약 10분 정도 더 끓입니다.\n\n6. **서빙하기**: 채식 부야베스가 완성되면 그릇에 담고, 다진 파슬리를 뿌린 후 바게트와 함께 서빙합니다.\n\n채식 부야베스는 다양한 채소와 식물성 단백질이 어우러져 풍부한 맛을 자랑합니다. 맛있게 드세요!')

#4.0

In [26]:
# fewShort는 모델에게 예제를 준다. > 더 나은 대답을 받기 위해 / 프롬프트의 system에서 ai에거 답변 요구사항을 올리는것보단, 예제로 출력 형식을 잡아주는게 더 효율적임

# 샘플들 > 이제 ai가 이러한 형태로 답변을 하게 됨

few_short_example = [
    {
        "question":"터키의 수도에 대해 알려줘.",
        "answer":"""
        터키의 수도는 '앙카라'입니다. 면적은 면적은 약 2516㎢이며, 인구는 2010년 기준 4,431,719명입니다.
        '앙카라'라는 지명은 '닻'을 의미하는 그리스어 앙퀴라에서 유래된 것으로 추측됩니다.
        '앙카라'에는 앙카라 성채, 아나톨리아 문명박물관, 악칼레, 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "question":"프랑스의 수도에 대해 알려줘.",
        "answer":"""
        프랑스의 수도는 '파리'입니다. 면적은 면적은 105.4㎢이며, 인구는 2023년 기준 13,024,518명입니다.
        '파리'라는 지명은 고대에 이 지역에 살았던 갈리아의 일족 파리시(Parisii)에서 유래되었습니다.
        '파리'에는 에펠탑, 루브루 박물관 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "question":"한국의 수도에 대해 알려줘.",
        "answer":"""
        한국의 수도는 '서울'입니다. 면적은 면적은 약 605.2㎢이며, 인구는 2024년 기준 9,360,400명입니다.
        '서울'라는 지명은 한국어의 순우리말로서 '한 나라의 수도(首都)', 곧 '국도(國都)'를 가리키는 일반명사입니다.
        '서울'에는 광하문, 국립중앙박물관, 경복궁, 등과 같은 관광지가 존재합니다.
        """
    },
]

In [27]:
# FewShotPromptTemplate는 PromptTemplate 기반으로 예제를 사용함

example_template ="""
    Human:{question}
    AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=few_short_example,
    suffix="Human: {country}의 수도에 대해 알려줘.",
    input_variables=["country"]
)

chain = prompt | CHAT
chain.invoke({"country":"아시리아"})

아시리아의 수도는 고대 도시 '니네베'입니다. 니네베는 현재의 이라크 북부에 위치하며, 티그리스 강 동쪽에 자리잡고 있습니다. 이 도시는 아시리아 제국의 전성기 동안 중요한 정치적, 문화적 중심지로 발전했습니다. 

니네베는 고대 아시리아의 왕궁과 신전, 그리고 장대한 도시 성벽으로 유명했습니다. 특히, 니네베 도서관은 아시리아의 문서와 문헌을 보관한 세계에서 가장 초기의 도서관 중 하나로 알려져 있습니다. 이 도시는 기원전 7세기경 아시리아 제국의 수도로서 번영했으나, 기원전 612년에 메디아와 바빌로니아의 연합군에 의해 멸망하게 됩니다. 오늘날 니네베는 고고학적 유적지로 남아 있으며, 그 역사적 중요성 때문에 많은 연구와 발굴이 이루어지고 있습니다.

AIMessageChunk(content="아시리아의 수도는 고대 도시 '니네베'입니다. 니네베는 현재의 이라크 북부에 위치하며, 티그리스 강 동쪽에 자리잡고 있습니다. 이 도시는 아시리아 제국의 전성기 동안 중요한 정치적, 문화적 중심지로 발전했습니다. \n\n니네베는 고대 아시리아의 왕궁과 신전, 그리고 장대한 도시 성벽으로 유명했습니다. 특히, 니네베 도서관은 아시리아의 문서와 문헌을 보관한 세계에서 가장 초기의 도서관 중 하나로 알려져 있습니다. 이 도시는 기원전 7세기경 아시리아 제국의 수도로서 번영했으나, 기원전 612년에 메디아와 바빌로니아의 연합군에 의해 멸망하게 됩니다. 오늘날 니네베는 고고학적 유적지로 남아 있으며, 그 역사적 중요성 때문에 많은 연구와 발굴이 이루어지고 있습니다.")

#4.2

In [28]:
few_short_chat_example = [
    {
        "country":"터키",
        "answer":"""
        터키의 수도는 '앙카라'입니다. 면적은 면적은 약 2516㎢이며, 인구는 2010년 기준 4,431,719명입니다.
        '앙카라'라는 지명은 '닻'을 의미하는 그리스어 앙퀴라에서 유래된 것으로 추측됩니다.
        '앙카라'에는 앙카라 성채, 아나톨리아 문명박물관, 악칼레, 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "country":"프랑스",
        "answer":"""
        프랑스의 수도는 '파리'입니다. 면적은 면적은 105.4㎢이며, 인구는 2023년 기준 13,024,518명입니다.
        '파리'라는 지명은 고대에 이 지역에 살았던 갈리아의 일족 파리시(Parisii)에서 유래되었습니다.
        '파리'에는 에펠탑, 루브루 박물관 등과 같은 관광지가 존재합니다.
        """
    },
    {
        "country":"한국",
        "answer":"""
        한국의 수도는 '서울'입니다. 면적은 면적은 약 605.2㎢이며, 인구는 2024년 기준 9,360,400명입니다.
        '서울'라는 지명은 한국어의 순우리말로서 '한 나라의 수도(首都)', 곧 '국도(國都)'를 가리키는 일반명사입니다.
        '서울'에는 광하문, 국립중앙박물관, 경복궁, 등과 같은 관광지가 존재합니다.
        """
    },
]

In [29]:
# FewShotChatMessagePromptTemplate은 ChatPromptTemplate을 기반으로하여 예제를 사용함

example_prompt = ChatPromptTemplate.from_messages([
        ("user","{country}의 수도에 대해 알려줘."),
        ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=few_short_chat_example,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","당신은 세계적인 지리 및 역사학자입니다. 특정 국가에 대해 물어보면 그게 얼마나 오래된 국가여도 대답해줄 수 있습니다."),
    example_prompt,
    ("human","{country}의 수도에 대해 알려줘.")
])

chain = final_prompt | CHAT
chain.invoke({"country": "몽골"})

몽골의 수도는 '울란바타르'입니다. 울란바타르는 몽골의 정치, 경제, 문화의 중심지로, 몽골에서 가장 큰 도시입니다. 면적은 약 4,700㎢이며, 인구는 2023년 기준으로 약 1,500,000명 이상입니다.

'울란바타르'라는 이름은 '붉은 영웅'이라는 뜻으로, 몽골의 독립 운동가인 수흐바타르(Sukhbaatar)의 이름에서 유래되었습니다. 이 도시는 고대 유목민의 정착지로 시작하여, 17세기에는 불교 사원이 세워지면서 발전하게 되었습니다.

울란바타르에는 성 바르하르 사원, 간단 사원, 그리고 몽골 국립 박물관 등 여러 관광지가 있습니다.

AIMessageChunk(content="몽골의 수도는 '울란바타르'입니다. 울란바타르는 몽골의 정치, 경제, 문화의 중심지로, 몽골에서 가장 큰 도시입니다. 면적은 약 4,700㎢이며, 인구는 2023년 기준으로 약 1,500,000명 이상입니다.\n\n'울란바타르'라는 이름은 '붉은 영웅'이라는 뜻으로, 몽골의 독립 운동가인 수흐바타르(Sukhbaatar)의 이름에서 유래되었습니다. 이 도시는 고대 유목민의 정착지로 시작하여, 17세기에는 불교 사원이 세워지면서 발전하게 되었습니다.\n\n울란바타르에는 성 바르하르 사원, 간단 사원, 그리고 몽골 국립 박물관 등 여러 관광지가 있습니다.")

#4.3

In [30]:
# 예시를 랜덤으로 선택해주는 ExampleSelector. BaseExampleSelector를 상속받음.
class ExampleSelector(BaseExampleSelector):

    def __init__(self,examples) -> None:
        self.examples = examples    # 시작할때, 샘플을 받음.

    def select_examples(self, input_variables): #파라미터로 받은 샘플을 고름
        from random import choice
        return [choice(self.examples)]
    
    def add_example(self, example):
        self.examples.append(example)

In [31]:
example_prompt = PromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=few_short_example,
    example_prompt=example_prompt,
    max_length=50
)

base_example_selector = ExampleSelector(
    examples=few_short_example
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    suffix="Human: {country}의 수도에 대해 알려줘.",
    input_variables=["country"],
    example_selector=base_example_selector
)

prompt.format(country="미국")


"\n    Human:한국의 수도에 대해 알려줘.\n    AI:\n        한국의 수도는 '서울'입니다. 면적은 면적은 약 605.2㎢이며, 인구는 2024년 기준 9,360,400명입니다.\n        '서울'라는 지명은 한국어의 순우리말로서 '한 나라의 수도(首都)', 곧 '국도(國都)'를 가리키는 일반명사입니다.\n        '서울'에는 광하문, 국립중앙박물관, 경복궁, 등과 같은 관광지가 존재합니다.\n        \n\n\nHuman: 미국의 수도에 대해 알려줘."

#4.4

In [32]:
# 인코딩 에러 발생해서 아래와 같은 코드로 구성

import json
from langchain.prompts import PromptTemplate

# JSON 파일을 UTF-8로 읽기
with open("../promt.json", "r", encoding="utf-8") as f:
    config = json.load(f)

# JSON 파일의 내용을 기반으로 PromptTemplate 생성
j_promt = PromptTemplate(
    input_variables=config["input_variables"],
    template=config["template"]
)

# 프롬프트 포맷 사용
formatted_output = j_promt.format(country="호주")
print(formatted_output)
CHAT.predict(formatted_output)

호주의 수도에 대해 알려줘.
호주의 수도는 캔버라(Canberra)입니다. 캔버라는 호주 연방 정부의 중심지로, 1913년에 설립되었습니다. 시드니와 멜버른 중간에 위치하고 있으며, 두 도시의 경쟁을 피하기 위해 선택된 장소입니다. 캔버라는 호주 국회의사당, 고등법원, 여러 정부 기관과 대사관들이 위치해 있어 정치적, 행정적 중심지 역할을 하고 있습니다. 또한, 호주 전쟁기념관, 국립미술관, 호주 국립박물관 등 다양한 문화시설도 있어 관광객들에게도 인기 있는 도시입니다.

'호주의 수도는 캔버라(Canberra)입니다. 캔버라는 호주 연방 정부의 중심지로, 1913년에 설립되었습니다. 시드니와 멜버른 중간에 위치하고 있으며, 두 도시의 경쟁을 피하기 위해 선택된 장소입니다. 캔버라는 호주 국회의사당, 고등법원, 여러 정부 기관과 대사관들이 위치해 있어 정치적, 행정적 중심지 역할을 하고 있습니다. 또한, 호주 전쟁기념관, 국립미술관, 호주 국립박물관 등 다양한 문화시설도 있어 관광객들에게도 인기 있는 도시입니다.'

In [33]:
# 인코딩 에러 발생해서 아래와 같은 코드로 구성

import yaml
from langchain.prompts import PromptTemplate

# YAML 파일을 UTF-8로 읽기
with open("../promt.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# YAML 파일의 내용을 기반으로 PromptTemplate 생성
y_promt = PromptTemplate(
    input_variables=config["input_variables"],
    template=config["template"]
)

# 프롬프트 포맷 사용
formatted_output = y_promt.format(country="호주")
print(formatted_output)
CHAT.predict(formatted_output)


호주의 수도에 대해 알려줘.
호주의 수도는 캔버라(Canberra)입니다. 캔버라는 호주 연방 정부의 중심지로, 1913년에 설립되었습니다. 호주에서 가장 큰 도시인 시드니와 멜버른의 중간에 위치하고 있으며, 두 도시 간의 경쟁을 피하기 위해 선택된 장소입니다. 캔버라는 계획 도시로, 국회의사당, 호주 전쟁 기념관, 여러 박물관과 갤러리 등이 있어 문화적인 중심지로도 알려져 있습니다.

'호주의 수도는 캔버라(Canberra)입니다. 캔버라는 호주 연방 정부의 중심지로, 1913년에 설립되었습니다. 호주에서 가장 큰 도시인 시드니와 멜버른의 중간에 위치하고 있으며, 두 도시 간의 경쟁을 피하기 위해 선택된 장소입니다. 캔버라는 계획 도시로, 국회의사당, 호주 전쟁 기념관, 여러 박물관과 갤러리 등이 있어 문화적인 중심지로도 알려져 있습니다.'

In [34]:
# 다양한 프롬프트를 하나로 묶는 방법으로 PipelinePromptTemplate이 있음
#탬플릿1
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)
#탬플릿2
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)
#탬플릿3
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)
#3개의 탬플릿을 합치기 위한 틀
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)
#prompts와 full_prompt을 통해 3개의 탬플릿을 묶음
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# chain 제작
chain = full_prompt | CHAT

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrr, me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a day o' plunderin' the high seas! Arg arg!

AIMessageChunk(content="Arrrr, me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a day o' plunderin' the high seas! Arg arg!")

#4.5

In [35]:
# set_llm_cache(InMemoryCache()) #캐시 메모리에 대화를 저장하는 코드.
# set_debug(True) #디버그 모드 활성화. 로그를 더 상세하게 볼 수 있음

# CHAT.predict("이탈리아 화덕 피자 만드는 방법")

In [36]:
CHAT.predict("이탈리아 화덕 피자 만드는 방법")

이탈리아 화덕 피자를 만드는 방법은 다음과 같습니다. 화덕 피자는 바삭하고 쫄깃한 도우와 신선한 재료가 특징입니다. 아래는 기본적인 화덕 피자 레시피입니다.

### 재료

#### 도우 재료
- 강력 밀가루 500g
- 물 325ml (미지근하게)
- 이스트 2g (또는 드라이 이스트 7g)
- 소금 10g
- 올리브유 10ml (선택 사항)

#### 토핑 재료
- 토마토 소스 (토마토 퓨레 또는 신선한 토마토)
- 모짜렐라 치즈 (신선한 것 또는 일반)
- 바질 잎
- 올리브유
- 소금과 후추 (기호에 따라)

### 만드는 방법

#### 1. 도우 만들기
1. **이스트 활성화**: 미지근한 물에 이스트를 넣고 잘 섞어 5~10분 정도 두어 활성화시킵니다.
2. **반죽하기**: 큰 볼에 밀가루와 소금을 넣고 잘 섞은 후, 활성화된 이스트와 물을 추가합니다. 반죽이 잘 섞일 때까지 손이나 주걱으로 저어줍니다.
3. **반죽 치대기**: 반죽이 하나로 뭉쳐지면, 깨끗한 작업대에 반죽을 옮겨 10분 정도 치대어 부드럽고 탄력 있는 반죽을 만듭니다.
4. **1차 발효**: 반죽을 기름을 바른 볼에 넣고 랩으로 덮은 후, 따뜻한 곳에서 1~2시간 발효시켜 반죽이 두 배로 부풀도록 합니다.

#### 2. 피자 조립
1. **오븐 예열**: 화덕 또는 오븐을 250도에서 예열합니다. 화덕이 있다면, 적절한 온도로 충분히 예열하세요.
2. **반죽 나누기**: 발효된 반죽을 꺼내어 2~4등분으로 나누고, 각 덩어리를 둥글게 만들어 줍니다.
3. **도우 펼치기**: 각 반죽을 손으로 또는 밀대로 얇게 펼쳐줍니다. 중앙은 두껍게, 가장자리는 얇게 만드는 것이 좋습니다.
4. **토핑 올리기**: 펼친 도우 위에 토마토 소스를 바르고, 모짜렐라 치즈를 고루 올립니다. 마지막으로 바질 잎과 올리브유를 뿌립니다.

#### 3. 굽기
1. **피자 굽기**: 예열된 화덕이나 오븐에 피자를 넣고 8~12분 정도 구워줍니다. 치즈가 녹고 가장자리가 바삭해질 때까지 굽습니

'이탈리아 화덕 피자를 만드는 방법은 다음과 같습니다. 화덕 피자는 바삭하고 쫄깃한 도우와 신선한 재료가 특징입니다. 아래는 기본적인 화덕 피자 레시피입니다.\n\n### 재료\n\n#### 도우 재료\n- 강력 밀가루 500g\n- 물 325ml (미지근하게)\n- 이스트 2g (또는 드라이 이스트 7g)\n- 소금 10g\n- 올리브유 10ml (선택 사항)\n\n#### 토핑 재료\n- 토마토 소스 (토마토 퓨레 또는 신선한 토마토)\n- 모짜렐라 치즈 (신선한 것 또는 일반)\n- 바질 잎\n- 올리브유\n- 소금과 후추 (기호에 따라)\n\n### 만드는 방법\n\n#### 1. 도우 만들기\n1. **이스트 활성화**: 미지근한 물에 이스트를 넣고 잘 섞어 5~10분 정도 두어 활성화시킵니다.\n2. **반죽하기**: 큰 볼에 밀가루와 소금을 넣고 잘 섞은 후, 활성화된 이스트와 물을 추가합니다. 반죽이 잘 섞일 때까지 손이나 주걱으로 저어줍니다.\n3. **반죽 치대기**: 반죽이 하나로 뭉쳐지면, 깨끗한 작업대에 반죽을 옮겨 10분 정도 치대어 부드럽고 탄력 있는 반죽을 만듭니다.\n4. **1차 발효**: 반죽을 기름을 바른 볼에 넣고 랩으로 덮은 후, 따뜻한 곳에서 1~2시간 발효시켜 반죽이 두 배로 부풀도록 합니다.\n\n#### 2. 피자 조립\n1. **오븐 예열**: 화덕 또는 오븐을 250도에서 예열합니다. 화덕이 있다면, 적절한 온도로 충분히 예열하세요.\n2. **반죽 나누기**: 발효된 반죽을 꺼내어 2~4등분으로 나누고, 각 덩어리를 둥글게 만들어 줍니다.\n3. **도우 펼치기**: 각 반죽을 손으로 또는 밀대로 얇게 펼쳐줍니다. 중앙은 두껍게, 가장자리는 얇게 만드는 것이 좋습니다.\n4. **토핑 올리기**: 펼친 도우 위에 토마토 소스를 바르고, 모짜렐라 치즈를 고루 올립니다. 마지막으로 바질 잎과 올리브유를 뿌립니다.\n\n#### 3. 굽기\n1. **피자 굽기**: 예열된 화덕이나 오븐에 피자를 넣고 8~12분 정

#4.6

In [37]:
with get_openai_callback() as usage: #usage를 통해 모델에서 사용한 토큰 수 및 비용 등의 데이터를 볼 수 있음
    CHAT.predict("와인을 만드는 방법")
    print(usage)

와인을 만드는 과정은 여러 단계로 이루어져 있습니다. 일반적으로 다음과 같은 단계로 진행됩니다:

1. **포도 수확**: 와인을 만들기 위해 가장 먼저 해야 할 일은 포도를 수확하는 것입니다. 포도의 품질과 수확 시기가 와인의 맛에 큰 영향을 미칩니다.

2. **포도 압착**: 수확한 포도를 압착하여 주스를 추출합니다. 이 과정에서 포도의 껍질, 씨앗, 과육이 분리됩니다. 레드 와인의 경우 껍질과 함께 발효되지만, 화이트 와인은 주스만 발효됩니다.

3. **발효**: 추출한 주스에 효모를 추가하여 발효를 시작합니다. 이 과정에서 당분이 알코올과 이산화탄소로 변환됩니다. 발효는 보통 몇 주에서 몇 달까지 걸릴 수 있습니다.

4. **숙성**: 발효가 끝난 후, 와인은 숙성 과정을 거칩니다. 이 과정에서 와인은 맛과 향이 더욱 깊어지고 부드러워집니다. 숙성은 오크통이나 스테인리스 스틸 탱크에서 이루어질 수 있습니다.

5. **필터링 및 병입**: 숙성이 끝난 후, 와인은 필터링을 통해 불순물을 제거하고, 병에 담아 저장합니다. 이때 추가적인 첨가물(예: 이산화황)을 넣어 와인의 보존성을 높일 수 있습니다.

6. **숙성 후 판매**: 병에 담긴 와인은 추가적인 숙성을 거칠 수 있으며, 이후 시장에 판매됩니다.

각 단계에서의 세부적인 방법과 기술은 와인의 종류와 생산자의 스타일에 따라 달라질 수 있습니다.Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


#5.0

In [38]:
# 메모리가 없으면 AI 모델은 이전의 대화를 기억하지 못한다. 그래서 우리는 memory를 사용해야한다.
#return_messages=True는 저장한 내용을 챗모델에게 사용 할 경우 True 아니면 False

#ConversationBufferMemory는 모든 대화 내용을 저장하는 방식
CB_memory = ConversationBufferMemory(return_messages=True)
# 모든 메모리는 save_context와 load_memory_variables를 갖고 있다. 각 메모리마다 작동 방식이 다를 뿐이다.
CB_memory.save_context({"input":"안녕!"},{"output":"안뇽!"})

CB_memory.load_memory_variables({})




{'history': [HumanMessage(content='안녕!'), AIMessage(content='안뇽!')]}

#5.1

In [39]:
#ConversationBufferWindowMemory는 대화의 특정 부분만을 저장하는 방식
CBW_memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input,output):
    CBW_memory.save_context({"input":input},{"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3) # k가 4라서 이거 이전의 메세지들은 저장되지 않음
add_message(4,4)
add_message(5,5)
add_message(6,6)
add_message(7,7)

CBW_memory.load_memory_variables({})

{'history': [HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6'),
  HumanMessage(content='7'),
  AIMessage(content='7')]}

#5.2

In [40]:
#ConversationSummaryMemory는 대화를 요약하여 저장하는 방식 / 초기에는 저장 공간이 많이 요구되지만, 이후에는 저장공간을 많이 사용되지 않음.
CS_memory = ConversationSummaryMemory(
    llm = CHAT
)

def add_message(input, output):
    CS_memory.save_context({"input": input}, {"output": output})

def get_history():
    return CS_memory.load_memory_variables({})

add_message("안녕 나는 유령개야. 오늘도 공부 중이야.", "와우! 완전 멋쪄")
get_history()

The human introduces themselves as a "ghost dog" and mentions they are studying today. The AI responds positively, expressing admiration.

{'history': 'The human introduces themselves as a "ghost dog" and mentions they are studying today. The AI responds positively, expressing admiration.'}

#5.3

In [51]:
#ConversationSummaryMemory는 특정 갯수의 메세지까지는 그냥 저장하고, 이후에 새로운 메세지에 의해 밀려나는 기존 메시지는 요약되어 저장됨
CSB_memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=100,
    return_messages=True,
)


def add_message(input, output):
    CSB_memory.save_context({"input": input}, {"output": output})


def get_history():
    return CSB_memory.load_memory_variables({})


add_message("안녕 나는 유령개야. 오늘도 공부 중이야.", "와우! 완전 멋쪄")
add_message("지금은 파이썬 공부중이야.", "와우! 파이썬 공부라니, 어떤걸 하고 있어?")
get_history()

The human greets the AI in Korean, introducing themselves as a ghost dog and mentioning they are studying today.

{'history': [SystemMessage(content='The human greets the AI in Korean, introducing themselves as a ghost dog and mentioning they are studying today.'),
  AIMessage(content='와우! 완전 멋쪄'),
  HumanMessage(content='지금은 파이썬 공부중이야.'),
  AIMessage(content='와우! 파이썬 공부라니, 어떤걸 하고 있어?')]}

#5.4

In [52]:
#ConversationKGMemory은 대화를 요약하면서 동시에 핵심적 키워드를 추출하는 방식
CKG_memory = ConversationKGMemory(
    llm=CHAT,
    return_messages=True,
)

def add_message(input, output):
    CKG_memory.save_context({"input": input}, {"output": output})


add_message("안녕 나는 유령개야. 오늘도 공부 중이야.", "와우! 완전 멋쪄")
add_message("지금은 파이썬 공부중이야.", "와우! 파이썬 공부라니, 어떤걸 하고 있어?")

NONE(NONE)

In [53]:
CKG_memory.load_memory_variables({"inputs": "지금 무엇을 공부하나요"})

NONE

{'history': []}

#5.5

In [54]:
memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=80,
    return_messages=True, # 이것도 있어야 작동하는듯?
    memory_key="chat_history" # 하단 탬플릿의 chat_history에 메모리에 저장된 내역을 전달하기 위한 설정값
)

# 메모리에 대화는 저장되나, 해당 로그가 기록이 llm에 전달되지 않고있음. 그래서, 아래와 같은 탬플릿 생성
template = """
    당신은 인간과 대화할때 도움이 되는 인공지능입니다.

    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=CHAT,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
#    verbose=True # 해당 값이 있으면 프롬프트가 실행될때 로그들을 확인 할 수 있음
)



chain.predict(question="내 이름은 유령개야!")

안녕하세요, 유령개님! 만나서 반가워요. 어떻게 도와드릴까요?

'안녕하세요, 유령개님! 만나서 반가워요. 어떻게 도와드릴까요?'

In [55]:
chain.predict(question="작명 사이트에서 이걸 추천해줬어!")

그렇군요, 유령개라는 이름 참 독특하고 멋있네요! 작명 사이트에서 추천해준 걸 선택하신 건가요? 그럼 그 이름에 어울리는 특별한 이야기나 의미가 있을지도 모르겠네요. 혹시 그 이름에 대해 더 알려주실 수 있나요? 함께 알아보면 더 재밌을 것 같아요!The human introduces themselves as "Ghost Dog." The AI greets them warmly and asks how it can help. Ghost Dog mentions that the name was recommended by a naming site. The AI finds the name unique and cool, suggesting that there may be a special story or meaning behind it. It encourages Ghost Dog to share more about the name, as exploring it together could be fun.

'그렇군요, 유령개라는 이름 참 독특하고 멋있네요! 작명 사이트에서 추천해준 걸 선택하신 건가요? 그럼 그 이름에 어울리는 특별한 이야기나 의미가 있을지도 모르겠네요. 혹시 그 이름에 대해 더 알려주실 수 있나요? 함께 알아보면 더 재밌을 것 같아요!'

In [56]:
chain.predict(question="내 이름이 뭔지 알아?")

안녕하세요, Ghost Dog님! 이름이 정말 독특하고 멋지네요. 그 이름에는 특별한 이야기나 의미가 있을 것 같아요. 이름에 대해 더 알려주시면 같이 탐험해보는 건 어떨까요? 함께 즐거운 시간을 보낼 수 있을 것 같아요.The human introduces themselves as "Ghost Dog" and the AI greets them warmly, finding the name unique and cool. The AI suggests that there may be a special story or meaning behind the name and encourages Ghost Dog to share more about it, as exploring it together could be fun. The human asks if the AI knows what their name means in Korean. The AI responds positively, expressing interest in learning more about the meaning behind the name and the story behind it.

'안녕하세요, Ghost Dog님! 이름이 정말 독특하고 멋지네요. 그 이름에는 특별한 이야기나 의미가 있을 것 같아요. 이름에 대해 더 알려주시면 같이 탐험해보는 건 어떨까요? 함께 즐거운 시간을 보낼 수 있을 것 같아요.'

In [ ]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='The human introduces themselves as "Ghost Dog." The AI greets Ghost Dog and asks how it can help. The human mentions a naming website and the AI finds the name recommendation interesting and asks about the website.'),
  HumanMessage(content='내 이름이 뭔지 알아?'),
  AIMessage(content='당신의 이름은 "Ghost Dog"이라고 소개하셨죠. 저는 여러분의 대화 상대가 될 수 있습니다. 무엇을 도와드릴까요?')]}

#5.6

In [ ]:
memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=120,
    return_messages=True,
    memory_key="chat_history" # 하단 탬플릿의 chat_history에 메모리에 저장된 내역을 전달하기 위한 설정값
)

prompt = ChatPromptTemplate.from_messages([
    ("system","당신은 인간과 대화할때 도움이 되는 인공지능입니다."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}"),
])

chain = LLMChain(
    llm=CHAT,
    memory=memory,
    prompt=prompt,
    verbose=True # 해당 값이 있으면 프롬프트가 실행될때 로그들을 확인 할 수 있음
)
chain.predict(question="내 이름은 유령개야!")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간과 대화할때 도움이 되는 인공지능입니다.
Human: 내 이름은 유령개야!
안녕, 유령개! 만나서 반가워. 무엇을 도와드릴까요?
> Finished chain.


'안녕, 유령개! 만나서 반가워. 무엇을 도와드릴까요?'

In [ ]:
chain.predict(question="너랑은 나눌 이야기 없어!")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간과 대화할때 도움이 되는 인공지능입니다.
Human: 내 이름은 유령개야!
AI: 안녕, 유령개! 만나서 반가워. 무엇을 도와드릴까요?
Human: 너랑은 나눌 이야기 없어!
알겠어요, 만약 도움이 필요하거나 궁금한 점이 생기면 언제든지 물어주세요. 저는 여러분을 돕기 위해 여기 있어요.
> Finished chain.
The human introduces themselves as "유령개." The AI responds with a greeting and asks how it can help.

'알겠어요, 만약 도움이 필요하거나 궁금한 점이 생기면 언제든지 물어주세요. 저는 여러분을 돕기 위해 여기 있어요.'

In [ ]:
chain.predict(question="난 게임을 좋아해. 특히 역사나 시뮬레이션. 아니면, 대전략류 게임을 좋아해")



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 인간과 대화할때 도움이 되는 인공지능입니다.
System: The human introduces themselves as "유령개." The AI responds with a greeting and asks how it can help.
Human: 너랑은 나눌 이야기 없어!
AI: 알겠어요, 만약 도움이 필요하거나 궁금한 점이 생기면 언제든지 물어주세요. 저는 여러분을 돕기 위해 여기 있어요.
Human: 난 게임을 좋아해. 특히 역사나 시뮬레이션. 아니면, 대전략류 게임을 좋아해
게임을 좋아하시는군요! 역사나 시뮬레이션, 대전략 게임을 즐기시는 것 같아요. 그런 게임들은 정말로 흥미로운 선택이죠. 어떤 게임을 좋아하시는지 더 자세히 알려주시면, 해당 장르의 다양한 게임을 추천해드릴 수 있어요. 혹시 좋아하는 게임이나 특정 장르의 게임이 있나요?
> Finished chain.
The human introduces themselves as "유령개." The AI responds with a greeting and offers assistance. The human expresses their love for games, especially historical, simulation, or grand strategy games. The AI acknowledges their interest in those genres and offers to recommend games based on their preferences.

'게임을 좋아하시는군요! 역사나 시뮬레이션, 대전략 게임을 즐기시는 것 같아요. 그런 게임들은 정말로 흥미로운 선택이죠. 어떤 게임을 좋아하시는지 더 자세히 알려주시면, 해당 장르의 다양한 게임을 추천해드릴 수 있어요. 혹시 좋아하는 게임이나 특정 장르의 게임이 있나요?'

#5.7

In [ ]:
memory = ConversationSummaryBufferMemory(
    llm=CHAT,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 인간과 대화할때 도움이 되는 인공지능입니다."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | CHAT


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [ ]:
invoke_chain("난 게임을 좋아해. 특히 역사나 시뮬레이션. 아니면, 대전략류 게임을 좋아해")

그렇군요, 게임을 좋아하시는군요! 역사나 시뮬레이션, 그리고 대전략 게임을 즐기시는 것 같네요. 그럼 Civilization 6 같은 게임을 플레이해보신 적이 있으신가요? Civilization 6는 역사적 배경을 기반으로 한 전략 시뮬레이션 게임으로, 플레이어는 문명을 발전시키고 세계를 정복하는 과정을 경험할 수 있어요. 여러 문명 중에서 자신이 좋아하는 문명이 있다면 어떤 문명인지 알려주실 수 있을까요? 함께 게임을 즐기는 것도 좋은 방법이 될 수 있겠죠. 함께 취미를 공유하면 더 재미있는 대화가 될 것 같아요.The human mentions enjoying games, particularly those related to history or simulation, or grand strategy games. The AI acknowledges the human's interest in games and asks if they have played Civilization 6, a strategy simulation game based on historical backgrounds. The AI explains the game's elements and suggests bonding over shared interests by discussing gaming together. The human asks if the AI remembers the game they mentioned playing frequently, which was Civilization 6. The AI recalls and elaborates on the game's features, inquires about the human's favorite civilization, and suggests gaming together as a way to connect. The human responds in Korean, expressing their love for games and histor

In [ ]:
invoke_chain("최근에는 문명6 많이 한 거 같아")

문명6는 역사적 배경을 바탕으로 한 전략 시뮬레이션 게임으로 유명하죠. 플레이어는 문명을 발전시키고 세계 정복을 목표로 하는 게임이죠. 문명6에서는 다양한 문명을 선택하여 자원을 수집하고 건물을 건설하며 전쟁을 벌이는 등의 다양한 요소가 있습니다. 문명6를 즐기는데 어떤 즐거움을 느끼시나요? 특히 선호하는 문명이 있다면 그 이유도 함께 알려주세요. 함께 게임에 대해 이야기해보는 건 어떨까요?The human mentions playing Civilization 6 frequently, a famous strategy simulation game based on historical backgrounds where players develop civilizations and aim for world domination. The AI describes the game's elements like choosing civilizations, collecting resources, building structures, and engaging in wars. The AI asks the human what enjoyment they find in playing Civilization 6, especially if they have a favorite civilization and why. The AI suggests discussing gaming together as a way to bond over shared interests.content='문명6는 역사적 배경을 바탕으로 한 전략 시뮬레이션 게임으로 유명하죠. 플레이어는 문명을 발전시키고 세계 정복을 목표로 하는 게임이죠. 문명6에서는 다양한 문명을 선택하여 자원을 수집하고 건물을 건설하며 전쟁을 벌이는 등의 다양한 요소가 있습니다. 문명6를 즐기는데 어떤 즐거움을 느끼시나요? 특히 선호하는 문명이 있다면 그 이유도 함께 알려주세요. 함께 게임에 대해 이야기해보는 건 어떨까요?'


In [ ]:
invoke_chain("내가 어떤 게임을 자주했다고 말했는지 기억해?")

네, 당신은 Civilization 6를 자주 플레이한다고 언급했었죠. Civilization 6는 역사적 배경을 기반으로 한 전략 시뮬레이션 게임으로, 플레이어들이 문명을 발전시키고 세계 정복을 목표로 하는 게임입니다. 게임에서는 다양한 문명을 선택하고 자원을 수집하며 건물을 건설하고 전쟁을 벌이는 등 다양한 요소들이 있죠. Civilization 6를 즐기는 이유와 가장 좋아하는 문명이 있다면 어떤 문명인지 알려주시겠어요? 함께 게임을 즐기는 것도 좋은 방법이 될 수 있습니다.The human mentions playing Civilization 6 frequently, a strategy simulation game based on historical backgrounds. The AI explains the game's elements and asks the human about their enjoyment and favorite civilization. The AI suggests bonding over shared interests by discussing gaming together. The human asks if the AI remembers what game they mentioned playing frequently. The AI recalls it was Civilization 6 and elaborates on the game's features. The AI inquires about the human's favorite civilization and suggests gaming together as a way to connect.content='네, 당신은 Civilization 6를 자주 플레이한다고 언급했었죠. Civilization 6는 역사적 배경을 기반으로 한 전략 시뮬레이션 게임으로, 플레이어들이 문명을 발전시키고 세계 정복을 목표로 하는 게임입니다. 게임에서는 다양한 문명을 선택하고 자원을 수집하며 건물을 건설하고 전쟁을

#6.1, #6.2

In [11]:
# CharacterTextSplitter는 하단의 RecursiveCharacterTextSplitter과 같은 page divider임. 같은 설정 값을 공유함 / 어느 특정 문자에서 문서를 자를지도 설정 가능
# from_tiktoken_encoder를 텍스트를 토큰으로 바꿔줌 > 즉, 사용자 및 개발자의 관점이 모델이 바라보는 관점과 동일해짐
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, # 덩어리의 크기를 제어 가능 / 글자수임
    chunk_overlap = 50, # 분할시 이전 문장 일부를 가져와서 자연스럽게? 만들어줌
  #  length_function=len, # 기본적으로 spiltter에서 분할한 단어 덩어리. 그러니까, 청크는 파이썬 기본 메소드인 len을 사용한다.
    separators="\n"
)

loader = TextLoader("../files/ch1.txt")

docs = loader.load()


# 파일 분할법
# 방법1
a = loader.load_and_split(text_splitter=splitter)

# 방법2
b = splitter.split_documents(docs)

In [12]:
len(b)

37

#6.3

embed는 단 하나의 토큰을 수십개의 특성으로. 마치 n차원 공간에 점을 찍는듯한 그런 느낌으로 토큰을 표현 및 수치화하는 것을 의미한다. 그리고, 각 단어들을 통해 새로운 단어에 대한 특성 및 벡터를 결정 할 수 있고, 각 단어들간의 관계를 알 수 있다. 이를 '단어대수학'라는 용어를 사용한다.

#6.4

In [42]:
embedder = OpenAIEmbeddings()

vector = embedder.embed_query("hi") # hi라는 단어를 임베딩했을때, 차원의 수는 1536이 나온다.
#즉, 해당 단어에 대한 특성이 1536개가 존재한다는 의미이다.

len(vector)

1536

In [43]:
embedder = OpenAIEmbeddings()

vector = embedder.embed_documents(["my name is","ghostdog"])
print(len(vector),len(vector[0]),len(vector[1]))

2 1536 1536


In [44]:
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200,
    chunk_overlap = 50,
    separators="\n"
)

loader = TextLoader("../files/test.txt")

docs = loader.load_and_split(text_splitter=splitter)

embedding = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embedding)

In [45]:
rs = vectorstore.similarity_search("the singularity")

len(rs)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


3

In [47]:
# 임베딩한 문서의 내용을 저장하기

cache_dir = LocalFileStore("../.cache/")

cache_embedings = CacheBackedEmbeddings.from_bytes_store(
    embedding, # 문서가 있는 임베딩
    cache_dir # 임베딩한 내용을 저장하기 위한 경로
)

#6.6

In [5]:
cache_dir = LocalFileStore("../.cache/") # 임베딩한 내용을 저장하기 위한 디랙토리

splitter = CharacterTextSplitter.from_tiktoken_encoder( # 원본 document를 토큰 단위로 쪼개주는 splitter
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/ch1.txt") # 파일 호출기

docs = loader.load_and_split(text_splitter=splitter) # 가져온 파일을 splitter로 쪼개고, dosc라는 변수에 저장

embeddings = OpenAIEmbeddings() # 임베딩 객체 생성

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir) 
# embeddings라는 객체를 통해 임베딩한 내용들을 cache_dir라는 경로에 저장해주는 cached_embeddings 객체를 생성


vectorstore = FAISS.from_documents(docs, cached_embeddings)
# cached_embeddings객체를 통해 docs라는 문서 토큰들?이 담긴 변수를 FAISS라는 vectorstore를 사용 및 활용하여 임베딩 한 내용을 저장

# RetrievalQA를 통해 토큰으로 쪼개고, 임베딩한 문서의 내용을 바탕으로 LLM에게 특정 방식(map_rerank)를 사용하여 질문하기.
# 모델은 문서의 내용을 바탕으로 사용자에게 대답함
# RetrievalQA는 아직까진 문서가 완벽하지 않음. 아직 연구 대상임
chain = RetrievalQA.from_chain_type( 
    llm=CHAT,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(), 
    # 위에선 vectorstore에 저장된 데이터를 사용했으나, 실제로는 임베딩된 데이터라면 종류 상관 없이 사용 가능하다.
)

chain.run("Describe Victory Mansions")

# RetrievalQA.from_chain_type에서 기본이 되는 체인 타입은 stuff이다. 

# stuff는 모든 문서를 한 번에 LLM에게 전달하여 답변을 생성합니다.
# 문서의 텍스트를 단순히 "stuff" (채워 넣기)한 후, LLM이 이를 바탕으로 질문에 대한 답변을 제공.

# Map Reduce는 각 문서에 대해 개별적으로 답변을 생성 후 모든 개별 답변을 종합하여 최종 답변을 생성.

# Map Rerank은 각 문서에 대해 개별적으로 답변을 생성, 각 답변에 대해 LLM이 점수를 부여. 가장 높은 점수를 받은 답변을 최종적으로 선택

# Refine Chain은 초기 문서에서 답변을 생성 후, 이후 남은 문서를 하나씩 추가로 제공하여 기존 답변을 개선하거나 보완하는 방식.

d:\coding-d\project\gpt\env\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


This document describes the Ministry of Love, not Victory Mansions
Score: 0Victory Mansions is a building with a gritty, unpleasant smell, featuring a large colored poster of a man's face with a caption that says "BIG BROTHER IS WATCHING YOU." The building has seven flights of stairs, with a non-functioning lift, and is described as part of the preparation for Hate Week. Inside the flat, there is a telescreen that cannot be completely shut off, and a fruity voice reading out figures related to pig-iron production.
Score: 90Victory Mansions is a building in London that has a roof from which you can see four other buildings, including the Ministry of Truth. It is where Winston stood and observed the surroundings. 
Score: 70Victory Mansions is described as having a living-room with an unusual position for the telescreen, shallow alcove, and a window. Winston is able to sit in the alcove and remain outside the range of the telescreen. The room has an unusual geography, and Winston's action

'Victory Mansions is a building with a gritty, unpleasant smell, featuring a large colored poster of a man\'s face with a caption that says "BIG BROTHER IS WATCHING YOU." The building has seven flights of stairs, with a non-functioning lift, and is described as part of the preparation for Hate Week. Inside the flat, there is a telescreen that cannot be completely shut off, and a fruity voice reading out figures related to pig-iron production.'

#6.8

In [12]:
cache_dir = LocalFileStore("../.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/ch1.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir) 
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. 
            Answer questions using only the following context. 
            If you don't know the answer just say you don't know, don't make it up:\n\n{context}""",
        ),
        ("human", "{question}"),
    ]
)



chain = {"context":retriver,"question":RunnablePassthrough()} | prompt | CHAT


chain.invoke("Describe Victory Mansions")

Victory Mansions is a building where Winston Smith resides. It has glass doors, a hallway that smells of boiled cabbage and old rag mats, and is located in a grimy landscape. The flat Winston lives in is seven flights up, with a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU" on each landing. Inside the flat, there is a telescreen that cannot be completely shut off, and the building is affected by an electric current cut-off during daylight hours as part of an economy drive in preparation for Hate Week.

AIMessageChunk(content='Victory Mansions is a building where Winston Smith resides. It has glass doors, a hallway that smells of boiled cabbage and old rag mats, and is located in a grimy landscape. The flat Winston lives in is seven flights up, with a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU" on each landing. Inside the flat, there is a telescreen that cannot be completely shut off, and the building is affected by an electric current cut-off during daylight hours as part of an economy drive in preparation for Hate Week.')

#6.9

In [7]:
cache_dir = LocalFileStore("../.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/ch1.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | CHAT


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = { "context": map_chain,"question":RunnablePassthrough() } | final_prompt | CHAT

chain.invoke("Describe Victory Mansions")

Victory Mansions is a run-down apartment building where Winston Smith, the protagonist of George Orwell's novel 1984, resides. The building is described as dilapidated and in poor condition, with shabby walls and a faulty elevator. The apartments are cramped and poorly maintained, reflecting the oppressive and dismal atmosphere of the society in which Winston lives.Victory Mansions is described as a building with glass doors that let in gritty dust along with Winston Smith as he enters. The hallway of Victory Mansions smells of boiled cabbage and old rag mats. There is a large colored poster on one end of the hallway depicting an enormous face of a man with a heavy black mustache and ruggedly handsome features. The building has seven flights of stairs, and the elevator is rarely working due to the current being cut off during daylight hours as part of an economy drive in preparation for Hate Week. On each landing, there is a poster with the caption "BIG BROTHER IS WATCHING YOU" beneath

AIMessageChunk(content='Victory Mansions is a run-down apartment building where Winston Smith resides in George Orwell\'s novel "1984." It is described as dilapidated, with shabby walls, cramped apartments, and a faulty elevator. The building has glass doors letting in gritty dust, a hallway smelling of boiled cabbage, and a large poster with the face of a man with a mustache. The building has seven flights of stairs, with posters saying "BIG BROTHER IS WATCHING YOU" on each landing. It offers a view of all four Ministries of Oceania and features an unusual living room with a telescreen positioned opposite the window.')